# 8.1 Extractive Summarization

**Installed the required Python prerequisite packages and libraries.**

In [ ]:
!pip install nltk
!pip install sumy
!pip install gensim
!pip install transformers

8.1.2 Implementing Extractive Summarization

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Sample text
text = """Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence
concerned with the interactions between computers and human language, in particular how to program computers to process
and analyze large amounts of natural language data. Challenges in natural language processing frequently involve speech
recognition, natural language understanding, and natural language generation."""

# Preprocess the text
sentences = sent_tokenize(text)
stop_words = set(stopwords.words('english'))

def preprocess_sentence(sentence):
    words = word_tokenize(sentence.lower())
    words = [word for word in words if word.isalnum() and word not in stop_words]
    return words

# Sentence scoring based on term frequency
def score_sentences(sentences):
    sentence_scores = []
    word_frequencies = FreqDist([word for sentence in sentences for word in preprocess_sentence(sentence)])

    for sentence in sentences:
        words = preprocess_sentence(sentence)
        sentence_score = sum(word_frequencies[word] for word in words)
        sentence_scores.append((sentence, sentence_score))

    return sentence_scores

# Select top-ranked sentences
def select_sentences(sentence_scores, num_sentences=2):
    sentence_scores.sort(key=lambda x: x[1], reverse=True)
    selected_sentences = [sentence[0] for sentence in sentence_scores[:num_sentences]]
    return selected_sentences

# Generate summary
sentence_scores = score_sentences(sentences)
summary_sentences = select_sentences(sentence_scores)
summary = ' '.join(summary_sentences)

print("Summary:")
print(summary)

8.1.3 Advanced Extractive Summarization Techniques

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import numpy as np
import networkx as nx

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# NLTK downloads (depending on the environment, 'punkt_tab' may also be required)
nltk.download('punkt')
nltk.download('stopwords')

# Sample text (replace with the reader's input)
text = """Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence
concerned with the interactions between computers and human language, in particular how to program computers to process
and analyze large amounts of natural language data. Challenges in natural language processing frequently involve speech
recognition, natural language understanding, and natural language generation."""

# 1) Sentences and light normalization
sentences = [s for s in sent_tokenize(text) if s.strip()]
stop_words = set(stopwords.words('english'))

def normalize(s):
    tokens = [w for w in word_tokenize(s.lower()) if w.isalnum() and w not in stop_words]
    return " ".join(tokens)

normalized = [normalize(s) for s in sentences]

# 2) TF-IDF vectorization per sentence
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(normalized)   # shape: (n_sentences, vocab_size)

# 3) Cosine similarity matrix
sim_matrix = cosine_similarity(X, X)
np.fill_diagonal(sim_matrix, 0.0)          # avoid dominant self-loops

# 4) TextRank with NetworkX (PageRank over the similarity graph)
graph = nx.from_numpy_array(sim_matrix)
scores = nx.pagerank(graph)

# 5) Selection of top-ranked sentences
num_sentences = 2
ranked = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
summary_sentences = [s for _, s in ranked[:num_sentences]]
summary = " ".join(summary_sentences)

print("Summary:")
print(summary)

In [ ]:
# Sample documents
documents = [
    "The cat sat on the mat",
    "The dog chased the ball",
    "The bird flew in the sky"
]

# Create a term-document matrix (one-hot encoded)
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
term_document_matrix = vectorizer.fit_transform(documents)

# Perform Singular Value Decomposition (SVD)
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=2)  # Reduce dimensionality to 2 for visualization
lsa_matrix = svd.fit_transform(term_document_matrix)

# Reduced term and document representations (topics)
terms = vectorizer.get_feature_names_out()
lsa_topics = svd.components_

# Print the results (example)
print("Terms:", terms)
print("Reduced Document Representations (Topics):")
print(lsa_matrix)
print("Reduced Term Representations (Topics):")
print(lsa_topics)

# 8.2 Abstractive Summarization

8.2.2 Implementing Abstractive Summarization

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the pre-trained BART model and tokenizer
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

# Sample text
text = """Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence
concerned with the interactions between computers and human language, in particular how to program computers to process
and analyze large amounts of natural language data. Challenges in natural language processing frequently involve speech
recognition, natural language understanding, and natural language generation."""

# Tokenize and encode the text
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

# Generate the summary
summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Summary:")
print(summary)

8.2.3 Advanced Abstractive Summarization Techniques

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the pre-trained T5 model and tokenizer
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Sample text
text = """Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence
concerned with the interactions between computers and human language, in particular how to program computers to process
and analyze large amounts of natural language data. Challenges in natural language processing frequently involve speech
recognition, natural language understanding, and natural language generation."""

# Tokenize and encode the text
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

# Generate the summary
summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Summary:")
print(summary)